<a href="https://colab.research.google.com/github/w95wayne10/personal_financial_management/blob/master/DH_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A hedger is short 10,000 European calls.  
As an option covers 100 shares of stock, N = 1,000,000.  


0 3 6 3

In [ ]:
import numpy as np

In [ ]:
# 2: 0: 0.41631 and 0.46255
path = [ 
 {0:(0.7225, 0.50397)},
 {-3:(0.1726,0.19247),0:(0.55274,0.50304),3:(1.39859,0.81218)},
 {-6:(0.0,0.0), -3:(0.0, 0.0), 0:(0.41631, 0.50304), 3:(1.22372, 1.0), 6:(2.46242, 1.0)},
 {-9:(0, 1.0), -6:(0,1.0), -3:(0, 1.0), 0:(0.0, 1.0), 3:(1.22372, 1.0), 6:(2.46242, 1.0), 9:(3.71628, 1.0)}
]

In [ ]:
#yt1 = yt0 + r - ht**2/2 + ht*epsilon
#Monte Carlo
S = 100
y0 = np.log(S)
yt = y0
h=np.sqrt(10.96/1e5)
#np.random.seed(1)
#print(np.exp(yt), h**2*1e5)
price_path = []
price_path.append(S)
for t in range(3):
  eps = np.random.standard_normal(1)[0]
  yt = yt - h**2/2 + h*eps
  h = np.sqrt(0.000006575 + 0.9*h**2 + 0.04*h**2*eps**2)
  price_path.append(np.exp(yt))
  #print(np.exp(yt), h**2*1e5, eps)
node_path = []
for idx, p in enumerate(price_path):
  a = idx*2+1
  b = a//2
  node_path.append(int((np.argmin(np.abs(p - np.exp(np.log(100)+(np.arange(a)-b)*3*np.sqrt(0.00006575)/2)))-b)*3))
node_path = np.array(node_path)
price_path = np.array(price_path)
node_path, price_path

(array([ 0,  0, -3,  0]),
 array([100.        ,  99.65756901,  99.25965841, 100.39932985]))

In [ ]:
#hedge
S = 100
N = 1000000
node_index = node_path.copy()
y = np.log(S)+node_index*np.sqrt(0.00006575)/2
S = np.exp(y)
temp = []
for idx, node in enumerate(node_index):
  temp.append(path[idx][node])
f,delta = list(zip(*temp))
f = np.array(f)
delta = np.array(delta)
change_delta = delta.copy()
change_delta[1:] = delta[1:]-delta[:-1]
shares_bought = N*change_delta
cost_shares = S*shares_bought
cumulative_cost = np.cumsum(cost_shares)
cumulative_cost[-1] - N*S[0]

608139.4574503154

In [ ]:
#hedge
S = 100
N = 1000000
node_index = node_path.copy()
y = np.log(S)+node_index*np.sqrt(0.00006575)/2
S = price_path#np.exp(y)
temp = []
for idx, node in enumerate(node_index):
  temp.append(path[idx][node])
f,delta = list(zip(*temp))
f = np.array(f)
delta = np.array(delta)
change_delta = delta.copy()
change_delta[1:] = delta[1:]-delta[:-1]
shares_bought = N*change_delta
cost_shares = S*shares_bought
cumulative_cost = np.cumsum(cost_shares)
cumulative_cost[-1] - N*S[0]

772069.7404237986

數量加大

In [ ]:
EPS = np.random.standard_normal((10,3))
EPS

array([[ 7.10436214e-01, -6.52583367e-01, -5.28164008e-01],
       [ 4.79986684e-02,  1.07499649e+00,  1.34140946e-02],
       [-5.89920712e-02, -1.27832653e+00,  5.99349652e-01],
       [-7.62483507e-01, -3.82454287e-01,  1.49873996e+00],
       [-1.00025489e-01, -1.19615318e+00, -5.92025021e-01],
       [ 2.11132642e-01, -6.11224176e-01,  2.74926756e+00],
       [ 7.92250316e-01, -1.21248355e+00, -2.73068305e-01],
       [ 4.06361085e-01,  2.95294368e+00, -1.11607832e-01],
       [-6.88656672e-01,  6.38653960e-01, -6.21110976e-01],
       [ 4.85118759e-01, -1.50563993e-03,  5.57120365e-01]])

In [ ]:
#yt1 = yt0 + r - ht**2/2 + ht*epsilon
#Monte Carlo
S = 100
y0 = np.log(S)
YT = np.zeros((10,4))
YT[:,0] = y0
h = np.sqrt(10.96/1e5)
H = np.zeros((10,4))
H[:,0] = h
np.random.seed(1)
#print(np.exp(yt), h**2*1e5)
PRICE_PATH = np.zeros((10,4))
PRICE_PATH[:,0] = S

In [ ]:
for t in range(3):
  YT[:,t+1] = YT[:,t]-H[:,t]**2/2 + H[:,t]*EPS[:,t]
  H[:,t+1] = np.sqrt(0.000006575 + 0.9*H[:,t]**2 + 0.04*H[:,t]**2*EPS[:,t]**2)
print(np.exp(YT), H**2*1e5, EPS,sep='\n')

[[100.         100.74100778 100.05653516  99.51102536]
 [100.         100.04477984 101.14878067 101.15739163]
 [100.          99.93278379  98.62570023  99.23701223]
 [100.          99.1994971   98.8011091  100.31900024]
 [100.          99.88986399  98.66631162  98.05773631]
 [100.         100.21578729  99.5836123  102.39632068]
 [100.         100.82733062  99.5596457   99.26933232]
 [100.         100.42082248 103.5143518  103.37111863]
 [100.          99.27619736  99.92976126  99.29090158]
 [100.         100.50365502 100.49675639 101.05919421]]
[[10.96       10.74276908 10.50899097 10.23285423]
 [10.96       10.52251002 10.61415887 10.21031937]
 [10.96       10.52302566 10.81605803 10.547366  ]
 [10.96       10.77637747 10.41929071 10.97102301]
 [10.96       10.52588624 10.73320774 10.4678638 ]
 [10.96       10.54104255 10.30196153 13.04394893]
 [10.96       10.79666639 11.00939399 10.59879178]
 [10.96       10.5938927  13.88710082 13.16281002]
 [10.96       10.72941033 10.48902129 10.

In [ ]:
#未完成
node_path = []
for idx, p in enumerate(price_path):
  a = idx*2+1
  b = a//2
  node_path.append(int((np.argmin(np.abs(p - np.exp(np.log(100)+(np.arange(a)-b)*3*np.sqrt(0.00006575)/2)))-b)*3))
node_path = np.array(node_path)
price_path = np.array(price_path)
node_path, price_path